# Grado en ciencia de datos - Big Data

# Práctica 2.1 - Introducción a SparkSQL

En esta práctica veremos las operaciones básicas para trabajar con los DataFrames de Spark. Esta primera práctica es una guía de todas las operaciones que se peuden realizar en SparkSQL con DataFrames. Prueba a cambiar los valores establecidos para comprobar su funcionamiento.

Ten en cuenta que una vez tengas en marcha Spark, podrás visualizar la evolución de cada trabajo de Spark en  <http://localhost:4040>

# Introducción a SparkSQL

Inicializar el `SparkSession`

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/12 13:05:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## API de SparkSQL
<http://spark.apache.org/docs/latest/api/python/pyspark.sql.html>

## Creación de un DataFrame
Podemos crear un DataFrame de 3 diferentes formas:

1. Infiriendo el esquema automáticamente a partir de los datos
1. Infiriendo el esquema automáticamente a partir de los metadatos
1. Definiendo explícitamente el esquema

A su vez, como con los RDDs podemos crearlo a partir de dos fuentes:

1. Cargando un conjunto de datos almacenado en un medio externo
2. Distribuyendo una colección de objetos existente

### Inferencia del esquema a partir de los datos

In [2]:
tuplas = [('Alice', 1), ('Bob', 4), ('Juan', 10), ('Pepe', 25), ('Panchito', 15)]

print("Sin nombre para las columnas: " + str(spark.createDataFrame(tuplas).collect()))
print("Con nombre para las columnas: " + str(spark.createDataFrame(tuplas, ['name', 'age']).collect()))

Sin nombre para las columnas: [Row(_1='Alice', _2=1), Row(_1='Bob', _2=4), Row(_1='Juan', _2=10), Row(_1='Pepe', _2=25), Row(_1='Panchito', _2=15)]
Con nombre para las columnas: [Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Pepe', age=25), Row(name='Panchito', age=15)]


In [3]:
rdd = sc.parallelize(tuplas)

print("Sin nombre para las columnas: " + str(spark.createDataFrame(rdd).collect()))
print("Con nombre para las columnas: " + str(spark.createDataFrame(rdd, ['name', 'age']).collect()))

Sin nombre para las columnas: [Row(_1='Alice', _2=1), Row(_1='Bob', _2=4), Row(_1='Juan', _2=10), Row(_1='Pepe', _2=25), Row(_1='Panchito', _2=15)]
Con nombre para las columnas: [Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Pepe', age=25), Row(name='Panchito', age=15)]


In [4]:
from pyspark.sql import Row

Person = Row('name', 'age') # Creamos una Row con los índices para poder crear filas con datos utilizándola
person = rdd.map(lambda r: Person(*r)) # con *r lo que hacemos es pasar la tupla como parámetro a la fila y crearla directamente con dichos datos
df2 = spark.createDataFrame(person)
df2.collect()

[Row(name='Alice', age=1),
 Row(name='Bob', age=4),
 Row(name='Juan', age=10),
 Row(name='Pepe', age=25),
 Row(name='Panchito', age=15)]

In [6]:
pandaDF = df2.toPandas()
print("DataFrame de Panda obtenido de un DataFrame Spark:")
print(pandaDF)
print("DataFrame de Spark creado a partir del Panda:")
print(spark.createDataFrame(pandaDF).collect())

DataFrame de Panda obtenido de un DataFrame Spark:
       name  age
0     Alice    1
1       Bob    4
2      Juan   10
3      Pepe   25
4  Panchito   15
DataFrame de Spark creado a partir del Panda:
[Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Pepe', age=25), Row(name='Panchito', age=15)]


### Especificación del esquema

In [7]:
from pyspark.sql.types import *
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)])
df3 = spark.createDataFrame(rdd, schema)
df3.collect()

[Row(name='Alice', age=1),
 Row(name='Bob', age=4),
 Row(name='Juan', age=10),
 Row(name='Pepe', age=25),
 Row(name='Panchito', age=15)]

In [8]:
print(spark.createDataFrame(rdd, "a: string, b: int").collect())

rdd2 = rdd.map(lambda row: row[1])
print(spark.createDataFrame(rdd2, "int").collect())

[Row(a='Alice', b=1), Row(a='Bob', b=4), Row(a='Juan', b=10), Row(a='Pepe', b=25), Row(a='Panchito', b=15)]
[Row(value=1), Row(value=4), Row(value=10), Row(value=25), Row(value=15)]


### Ejemplo

In [9]:
import datetime
data = [
    Row(
        name="Alex",
        num_pets=3,
        paid_in_full=True,
        preferences={
            "preferred_vet": "Dr. Smith",
            "preferred_appointment_day": "Monday"
        },
        registered_on=datetime.datetime(2015, 1, 1, 12,0),
        visits=[
            datetime.datetime(2015, 2, 1, 11, 0),
            datetime.datetime(2015, 2, 2, 10, 45),
        ],
    ), 
    Row(
        name="Charlie",
        num_pets=1,
        paid_in_full=True,
        preferences={},
        registered_on=datetime.datetime(2016, 5, 1, 12,0),
        visits=[],
    ),



    Row(
        name="Beth",
        num_pets=2,
        paid_in_full=False,
        preferences={
            "preferred_vet": "Dr. Travis",
        },
        registered_on=datetime.datetime(2013, 1, 1, 12,0),
        visits=[
            datetime.datetime(2015, 1, 15, 12, 15),
            datetime.datetime(2015, 2, 1, 11, 15),
        ],
    ),
]

### Inferencia del esquema a partir de los datos

In [10]:
# Creamos un RDD con los datos de ejemplo
dataRDD = sc.parallelize(data)

# Creamos un DataFrame a partir del RDD, infiriendo el esquema a partir de la primera Row
print("RDD: Esquema inferido a partir de la primera fila.")

# Por defecto samplingRatio=None
dataDF = spark.createDataFrame(dataRDD, samplingRatio=None)
dataDF.printSchema()

# Creamos un DataFrame a partir del RDD, infiriendo el esquema a partir de un sampling de las Rows
print("RDD: Esquema inferido de un sampling aleatorio.")
dataDF = spark.createDataFrame(dataRDD, samplingRatio=0.6)
dataDF.printSchema()

RDD: Esquema inferido a partir de la primera fila.
root
 |-- name: string (nullable = true)
 |-- num_pets: long (nullable = true)
 |-- paid_in_full: boolean (nullable = true)
 |-- preferences: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- registered_on: timestamp (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)

RDD: Esquema inferido de un sampling aleatorio.
root
 |-- name: string (nullable = true)
 |-- num_pets: long (nullable = true)
 |-- paid_in_full: boolean (nullable = true)
 |-- preferences: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- registered_on: timestamp (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)



### Especificación del esquema

In [11]:
from pyspark.sql.types import ArrayType, BooleanType, DateType, \
    IntegerType, MapType, StringType, TimestampType, StructField, StructType

schema = StructType(
    [
        StructField("name", StringType(), True),
        StructField("num_pets", IntegerType(), True),
        StructField("paid_in_full", BooleanType(), True),
        StructField("preferences", MapType(StringType(), StringType(), True), True),
        StructField("registered_on", DateType(), True),
        StructField("visits", ArrayType(TimestampType(), True), True),
    ]
)

# Crear un DataFrame a partir de un RDD, especificando el esquema
print("RDD: Esquema espcificado explícitamente.")
dataDF = spark.createDataFrame(dataRDD, schema)
dataDF.printSchema()

RDD: Esquema espcificado explícitamente.
root
 |-- name: string (nullable = true)
 |-- num_pets: integer (nullable = true)
 |-- paid_in_full: boolean (nullable = true)
 |-- preferences: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- registered_on: date (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)



### Lectura del DataFrame desde un fichero JSON

In [12]:
# Crear un DataFrame a partir de un fichero JSON, infiriendo el esquema a partir de todas las filas
print("JSON: Esquema inferido a partir de todas las filas.")
dataDF = spark.read.option("samplingRatio", 1.0).json("datos/data.json")
dataDF.printSchema()

JSON: Esquema inferido a partir de todas las filas.
root
 |-- name: string (nullable = true)
 |-- num_pets: long (nullable = true)
 |-- paid_in_full: boolean (nullable = true)
 |-- preferences: struct (nullable = true)
 |    |-- preferred_appointment_day: string (nullable = true)
 |    |-- preferred_vet: string (nullable = true)
 |-- registered_on: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
# Crear un DataFrame a partir de un fichero JSON, especificando el esquema
print("JSON: Esquema especificado explícitamente.")
dataDF = spark.read.json("datos/data.json", schema)
dataDF.printSchema()

JSON: Esquema especificado explícitamente.
root
 |-- name: string (nullable = true)
 |-- num_pets: integer (nullable = true)
 |-- paid_in_full: boolean (nullable = true)
 |-- preferences: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- registered_on: date (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: timestamp (containsNull = true)



## Acceso a las columnas de un DataFrame

In [14]:
print(df2.age)
print(df2["age"])
print((df2.age + 1).alias("cumpleaños"))

Column<'age'>
Column<'age'>
Column<'(age + 1) AS `cumpleaños`'>


# Operaciones con DataFrames

Hay dos tipos de operaciones sobre DataFrames, como los RDDs:
1. Transformaciones: Crean un nuevo DataFrame a partir de otro **(EVALUACIÓN VAGA)** hasta que no se ejecuta una acción no se realiza la transformación
2. Acciones: Utilizan el DataFrame para lograr un resultado, es recibido por el driver o escrita en disco

## Transformaciones sobre DataFrames

### Transformaciones básicas


Transformación | Descripción
------------- | -------------
*select(*\**cols)* | Devuelve un nuevo DataFrame proyectando una serie de expresiones que pueden ser nombres de columnas o expresiones de tipo Column. Si se utiliza "\*", todas las columnas del DataFrame se proyectan al nuevo DataFrame
*selectExpr(*\**expr)* | Variante de select que admite expresiones SQL
*filter(condition) / where(condition)* | Filtra las filas usando la condición especificada
*orderBy(*\**cols, ascending)* | Devuelve un nuevo DataFrame ordenado por las columnas especificadas. Por defecto en orden ascendente
*sort(*\**cols, *\*\**kwargs)* | Devuelve un DataFrame ordenado por las columnas especificadas. La condición debe ser tipo Column o expresión SQL

In [15]:
# DataFrame de ejemplo
tuplas = [('Alice', 1), ('Bob', 4), ('Juan', 10), ('Pepe', 25), ('Panchito', 15)]
df = spark.createDataFrame(tuplas, ['name', 'age'])

### `select(*cols)`

In [16]:
print(df.select('*').collect())

print(df.select('name', 'age').collect())

print(df.select(df.name, (df.age + 10).alias('age')).collect())

[Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Pepe', age=25), Row(name='Panchito', age=15)]
[Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Pepe', age=25), Row(name='Panchito', age=15)]
[Row(name='Alice', age=11), Row(name='Bob', age=14), Row(name='Juan', age=20), Row(name='Pepe', age=35), Row(name='Panchito', age=25)]


### `selectExpr(*expr)`

In [17]:
df.selectExpr("age * 2", "abs(age)").collect()

[Row((age * 2)=2, abs(age)=1),
 Row((age * 2)=8, abs(age)=4),
 Row((age * 2)=20, abs(age)=10),
 Row((age * 2)=50, abs(age)=25),
 Row((age * 2)=30, abs(age)=15)]

### `filter(condition) = where(condition)`

In [18]:
# Modo programación orientada a objetos (POO)
print("Modo POO, edad > 18: " + str( df.filter(df.age > 18).collect() ))

print("Modo POO, edad == 1: " + str( df.where(df.age == 1).collect() ))

# Modo SQL
print("Modo SQL, edad > 18: " + str( df.filter("age > 18").collect() ))

print("Modo SQL, edad = 1: " + str( df.where("age = 1").collect() ))

Modo POO, edad > 18: [Row(name='Pepe', age=25)]
Modo POO, edad == 1: [Row(name='Alice', age=1)]
Modo SQL, edad > 18: [Row(name='Pepe', age=25)]
Modo SQL, edad = 1: [Row(name='Alice', age=1)]


### `orderBy(*cols, ascending)` y `sort(*cols, **kwargs)`

Diferentes argumentos, pero el resultado es el mismo. En ambas, se reciben primero las columnas por las que ordenar el DataFrame y después la forma en la que se quiere ordenar.

In [19]:
print("Ordenar por edad descendente:")
print(df.sort(df.age.desc()).collect())
print(df.sort("age", ascending=False).collect())
print(df.orderBy(df.age.desc()).collect())

from pyspark.sql.functions import *

print("Ordenar por edad ascendente:")
print(df.sort(asc("age")).collect())

print("Ordenar por edad descendente y nombre ascendente:")
print(df.orderBy(desc("age"), "name").collect())
print(df.orderBy(["age", "name"], ascending=[0, 1]).collect())

Ordenar por edad descendente:
[Row(name='Pepe', age=25), Row(name='Panchito', age=15), Row(name='Juan', age=10), Row(name='Bob', age=4), Row(name='Alice', age=1)]
[Row(name='Pepe', age=25), Row(name='Panchito', age=15), Row(name='Juan', age=10), Row(name='Bob', age=4), Row(name='Alice', age=1)]
[Row(name='Pepe', age=25), Row(name='Panchito', age=15), Row(name='Juan', age=10), Row(name='Bob', age=4), Row(name='Alice', age=1)]
Ordenar por edad ascendente:
[Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Panchito', age=15), Row(name='Pepe', age=25)]
Ordenar por edad descendente y nombre ascendente:
[Row(name='Pepe', age=25), Row(name='Panchito', age=15), Row(name='Juan', age=10), Row(name='Bob', age=4), Row(name='Alice', age=1)]
[Row(name='Pepe', age=25), Row(name='Panchito', age=15), Row(name='Juan', age=10), Row(name='Bob', age=4), Row(name='Alice', age=1)]


### Transformaciones adicionales


Transformación | Descripción
------------- | -------------
*distinct()* | Devuelve un nuevo DataFrame con las filas únicas del original
*dropDuplicates(*\**cols)* | Devuelve un nuevo DataFrame sin filas duplicadas considerando las columnas especificadas
*withColumn(colName, col)* | Devuelve un nuevo DataFrame añadiendo una nueva columna o reemplazando la columna existente con el mismo nombre
*withColumnRenamed(existing, new)* | Devuelve un DataFrame renombrando una columna existente
*drop(col)* | Devuelve un nuevo DataFrame con la columna especificada eliminada
*limit(num)* | Limita el número de filas obtenidas como resultado
*cache()* | Mantiene el DataFrame almacenado en memoria para ser reusado


### `distinct()` y `dropDuplicates(*cols)`
La diferencia está en que en `dropDuplicates` podemos especificar los campos por los cuales decidimos que dos filas están repetidas.

In [20]:
from pyspark.sql import Row
df2 = sc.parallelize([ \
    Row(name='Alice', age=5, height=80), \
    Row(name='Alice', age=5, height=80), \
    Row(name='Alice', age=10, height=80)]).toDF()

print("DataFrame:")
df2.show()

print("distinct(): ")
df2.distinct().show()

print("dropDuplicates(): ")
df2.dropDuplicates().show() 

print("dropDuplicates('name', 'height')")
df2.dropDuplicates(['name', 'height']).show()

DataFrame:
+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
|Alice|  5|    80|
|Alice| 10|    80|
+-----+---+------+

distinct(): 
+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
|Alice| 10|    80|
+-----+---+------+

dropDuplicates(): 
+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
|Alice| 10|    80|
+-----+---+------+

dropDuplicates('name', 'height')
+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
+-----+---+------+



### `drop(col) `
Recordad que drop elimina la columna pero devuelve un nuevo DataFrame, no modifica el que tenemos (son inmutables)

In [21]:
print("Eliminamos la columna age:")
print(df.drop('age').collect())
print(df.drop(df.age).collect())

Eliminamos la columna age:
[Row(name='Alice'), Row(name='Bob'), Row(name='Juan'), Row(name='Pepe'), Row(name='Panchito')]
[Row(name='Alice'), Row(name='Bob'), Row(name='Juan'), Row(name='Pepe'), Row(name='Panchito')]


### `limit(num)`

In [22]:
print("Limit(1):" + str( df.limit(1).collect() ))
print("Limit(0):" + str( df.limit(0).collect() ))

Limit(1):[Row(name='Alice', age=1)]
Limit(0):[]


### `withColumn(colName, col) `

In [23]:
print("Añadimos una columna denominada age2 obtenida a partir de la columna age y sumando un 2:")
df.withColumn('age2', df.age + 2).collect()

Añadimos una columna denominada age2 obtenida a partir de la columna age y sumando un 2:


[Row(name='Alice', age=1, age2=3),
 Row(name='Bob', age=4, age2=6),
 Row(name='Juan', age=10, age2=12),
 Row(name='Pepe', age=25, age2=27),
 Row(name='Panchito', age=15, age2=17)]

### `withColumnRenamed(existing, new) `

In [24]:
print("Renombramos la columna age a age2 y obtenemos un nuevo DataFrame con el cambio:")
df.withColumnRenamed('age', 'age2').collect()

Renombramos la columna age a age2 y obtenemos un nuevo DataFrame con el cambio:


[Row(name='Alice', age2=1),
 Row(name='Bob', age2=4),
 Row(name='Juan', age2=10),
 Row(name='Pepe', age2=25),
 Row(name='Panchito', age2=15)]

### Transformaciones: Operaciones con columnas


Operación | Descripción
------------- | -------------
*alias(*\**alias)* | Devuelve la columna con un nuevo nombre
*between(lowerBound, upperBound)* | True si el valor está entre los dos valores
*isNull() / isNotNull()* | True si el valor es nulo y viceversa
*when(condition, value) / otherwise(value)* | Evalúa una lista de condiciones y en base a estas devuelve un valor u otro
*Startswith(other), substring(startPos, len), like(otheR)* | Funciones para operar con strings
*isin(*\**cols)* | True si el valor está en la lista de argumentos
*explode(col)* | Devuelve una nueva fila para cada elemento en el array
*lit(value)* | Crea una columna con el valor literal
*length(col)* | Longitud de la columna

Estas operaciones por si mismas solo devuelve una columna (expresión SQL). Para poder aplicalas al DataFrame se deben utilizar en conjunto con una transformación tipo `select`.

### `alias(*alias)`

In [25]:
print("Columna cambio de nombre: ")
print(df.name.alias("hola"))

print("\nDataFrame con columna nombre renombrada a hola: ")
df.select(df.name.alias("hola")).show() # Solo aparece la columna seleccionada!

Columna cambio de nombre: 
Column<'name AS hola'>

DataFrame con columna nombre renombrada a hola: 
+--------+
|    hola|
+--------+
|   Alice|
|     Bob|
|    Juan|
|    Pepe|
|Panchito|
+--------+



### `between(lowerBound, upperBound) `

In [26]:
print("Columna con condición between: ")
print(df.age.between(18, 65))
print("\nDataFrame con nombre y columna con True o False según se cumple la condición")
df.select(df.name, df.age.between(2, 4)).show()

Columna con condición between: 
Column<'((age >= 18) AND (age <= 65))'>

DataFrame con nombre y columna con True o False según se cumple la condición
+--------+---------------------------+
|    name|((age >= 2) AND (age <= 4))|
+--------+---------------------------+
|   Alice|                      false|
|     Bob|                       true|
|    Juan|                      false|
|    Pepe|                      false|
|Panchito|                      false|
+--------+---------------------------+



### `isNull() / isNotNull() `

In [27]:
print("Expresión Columna con isNull()/isNotNull():")
print(df.age.isNull())
print(df.age.isNotNull())

print("\nDataFrame con filas que tienen el nombre nulo")
df.filter(df.name.isNull()).show()

Expresión Columna con isNull()/isNotNull():
Column<'(age IS NULL)'>
Column<'(age IS NOT NULL)'>

DataFrame con filas que tienen el nombre nulo
+----+---+
|name|age|
+----+---+
+----+---+



### `when(condition, value) / otherwise(value) `

In [28]:
from pyspark.sql import functions as F
print("Condición when/otherwise:")
print(F.when(df.age > 18, "adulto").otherwise("peque"))

print("\nEjemplos when/otherwise")
df.select(df.name, F.when(df.age > 3, 1).otherwise(0)).show()
df.select(df.name, F.when(df.age > 4, 1).when(df.age < 3, -1).otherwise(0)).show()

Condición when/otherwise:
Column<'CASE WHEN (age > 18) THEN adulto ELSE peque END'>

Ejemplos when/otherwise
+--------+-------------------------------------+
|    name|CASE WHEN (age > 3) THEN 1 ELSE 0 END|
+--------+-------------------------------------+
|   Alice|                                    0|
|     Bob|                                    1|
|    Juan|                                    1|
|    Pepe|                                    1|
|Panchito|                                    1|
+--------+-------------------------------------+

+--------+------------------------------------------------------------+
|    name|CASE WHEN (age > 4) THEN 1 WHEN (age < 3) THEN -1 ELSE 0 END|
+--------+------------------------------------------------------------+
|   Alice|                                                          -1|
|     Bob|                                                           0|
|    Juan|                                                           1|
|    Pepe|       

### `Startswith(other), substring(startPos, len), like(otheR) `

In [29]:
print("Columna like: ")
print(df.name.like("M*"))

print("\nDataFrame con primeras tres letras de los nombres")
df.select(df.name.substr(1, 3).alias("Inicial")).collect()

Columna like: 
Column<'name LIKE M*'>

DataFrame con primeras tres letras de los nombres


[Row(Inicial='Ali'),
 Row(Inicial='Bob'),
 Row(Inicial='Jua'),
 Row(Inicial='Pep'),
 Row(Inicial='Pan')]

### `isin(*cols) `

In [30]:
print("Columna con expresión isin()")
print(df.name.isin("Mikel", "Pepe"))

# Ojo! Estamos usando la operación para filtrar las filas, para ello podemos usar df[condición]!
print("\nFiltrar filas con nombres Bob y Mikel")
print(df[df.name.isin("Bob", "Mikel")].collect())
print("Filtrar filas con edades 1, 2 o 3")
print(df[df.age.isin([1, 2, 3])].collect())

Columna con expresión isin()
Column<'(name IN (Mikel, Pepe))'>

Filtrar filas con nombres Bob y Mikel
[Row(name='Bob', age=4)]
Filtrar filas con edades 1, 2 o 3
[Row(name='Alice', age=1)]


### `explode(col) `
Explode puede utilizarse para simular el flatMap de los RDDs, ya que crea tantas filas como valores tenga un array o map

In [31]:
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
print("DataFrame original")
print(eDF.collect())

print("\nDataFrame con filas expandidas según el array de enteros")
eDF.select(F.explode(eDF.intlist).alias("anInt")).show()

print("DataFrame con filas expandidas según el map de caracteres")
eDF.select('*', F.explode(eDF.mapfield).alias("key", "value")).show()

DataFrame original
[Row(a=1, intlist=[1, 2, 3], mapfield={'a': 'b'})]

DataFrame con filas expandidas según el array de enteros
+-----+
|anInt|
+-----+
|    1|
|    2|
|    3|
+-----+

DataFrame con filas expandidas según el map de caracteres
+---+---------+--------+---+-----+
|  a|  intlist|mapfield|key|value|
+---+---------+--------+---+-----+
|  1|[1, 2, 3]|{a -> b}|  a|    b|
+---+---------+--------+---+-----+



### `lit(value)`

In [32]:
from pyspark.sql import functions as F
print("Columna de unos: ")
print(F.lit(1))

print("\nDataFrame con una nueva columna con unos:")
df.select("*", F.lit(1)).show()

Columna de unos: 
Column<'1'>

DataFrame con una nueva columna con unos:
+--------+---+---+
|    name|age|  1|
+--------+---+---+
|   Alice|  1|  1|
|     Bob|  4|  1|
|    Juan| 10|  1|
|    Pepe| 25|  1|
|Panchito| 15|  1|
+--------+---+---+



### `length(col) `

In [33]:
print("Columna para obtener longitud del texto: " )
print(F.length(df.name))

print("\nDataFrame con la longitud de cada nombre")
df.select(F.length(df.name).alias('len')).show()

Columna para obtener longitud del texto: 
Column<'length(name)'>

DataFrame con la longitud de cada nombre
+---+
|len|
+---+
|  5|
|  3|
|  4|
|  4|
|  8|
+---+



### Transformaciones con pseudo-conjuntos
Aunque no los hemos visto en teoría, podemos hacer operaciones entre pseudo-conjuntos al igual que con los RDDs

Transformación | Descripción
------------- | -------------
*distinct()* | Devuelve el DataFrame sin elementos repetidos – ¡Cuidado! Requiere shuffle (enviar datos por red)
*union(rdd)* | Devuelve la unión de los elementos en los dos DataFrame  (se mantienen los duplicados)
*intersect(rdd)* | Devuelve la instersección de los elementos en los dos DataFrame (elimina los duplicados) – ¡Cuidado! Requiere shuffle (datos por red)
*subtract(rdd)* | Devuelve los elementos presentes en el primer DataFrame y no en el segundo – ¡Cuidado! También requiere de shuffle

In [34]:
df1 = spark.createDataFrame(["agua", "vino", "cerveza", "agua", "agua", "vino"], "string")
df2 = spark.createDataFrame(["cerveza", "cerveza", "agua", "agua", "vino", "coca-cola", "naranjada"], "string")

print("distinct: " + str(df1.distinct().collect()))
print("union: " + str(df1.union(df2).collect()))
print("intersect: " + str(df1.intersect(df2).collect()))
print("substract: " + str(df1.subtract(df2).collect()))

distinct: [Row(value='agua'), Row(value='cerveza'), Row(value='vino')]
union: [Row(value='agua'), Row(value='vino'), Row(value='cerveza'), Row(value='agua'), Row(value='agua'), Row(value='vino'), Row(value='cerveza'), Row(value='cerveza'), Row(value='agua'), Row(value='agua'), Row(value='vino'), Row(value='coca-cola'), Row(value='naranjada')]
intersect: [Row(value='agua'), Row(value='cerveza'), Row(value='vino')]
substract: []


## Acciones sobre RDDs

### Acciones básicas


Acción | Descripción
------------- | -------------
*show(n=20, truncate=True)* | Imprime las primeras n filas del DataFrame. Truncate indica si se quiere truncar los strings demasiado largos
*count()* | Devuelve el número de filas en el DataFrame
*collect()* | Devuelve todas las filas del DataFrame como una lista de Rows **Cuidado: Debe de caber en memoria**
*first()* | Devuelve la primera fila del DataFrame
*take(n)* | Devuelve las primeras n filas del DataFrame como lista de Rows
*toPandas()* | Devuelve el contenido del DataFrame como un pandas.DataFrame **Cuidado: Debe de caber en memoria**
*columns* | Devuelve todos los nombres de columnas como una lista
*describe(*\**cols)* | Calcula estadísticas para las columnas numéricas  (count, mean, stddev, min, y max)
*explain(extended=False)* | Imprime los planes físicos y lógicos para debugging

In [35]:
print("DataFrame show(): ")
df.show()

print("Dos primeras filas con take(2): " + str(df.take(2)))

print("\nPrimera fila con first(): " + str(df.first()))

print("\nDataFrame completo con collect(): " + str( df.collect() ))

print("\nNúmero de filas en el DataFrame con count(): " + str( df.count() ) )

print("\nDataFrame como panda: ")
print(df.toPandas())

print("\nColumnas en el DataFrame con columns: "+ str( df.columns ))

print("\nPlanes físicos y lógicos con explain:")
df.filter(df.age > 10).select(df.age).explain(True)

DataFrame show(): 
+--------+---+
|    name|age|
+--------+---+
|   Alice|  1|
|     Bob|  4|
|    Juan| 10|
|    Pepe| 25|
|Panchito| 15|
+--------+---+

Dos primeras filas con take(2): [Row(name='Alice', age=1), Row(name='Bob', age=4)]

Primera fila con first(): Row(name='Alice', age=1)

DataFrame completo con collect(): [Row(name='Alice', age=1), Row(name='Bob', age=4), Row(name='Juan', age=10), Row(name='Pepe', age=25), Row(name='Panchito', age=15)]

Número de filas en el DataFrame con count(): 5

DataFrame como panda: 
       name  age
0     Alice    1
1       Bob    4
2      Juan   10
3      Pepe   25
4  Panchito   15

Columnas en el DataFrame con columns: ['name', 'age']

Planes físicos y lógicos con explain:
== Parsed Logical Plan ==
Project [age#107L]
+- Filter (age#107L > cast(10 as bigint))
   +- LogicalRDD [name#106, age#107L], false

== Analyzed Logical Plan ==
age: bigint
Project [age#107L]
+- Filter (age#107L > cast(10 as bigint))
   +- LogicalRDD [name#106, age#107L], f

### Transformaciones para realizar agregaciones



Transformación | Descripción
------------- | -------------
*agg(*\**exprs)* | Realiza agregaciones sobre el DataFrame completo sin agrupar (equivalente a df.groupBy.agg()) – Agregaciones disponibles: avg, max, min, sum, count. exprs puede ser un diccionario clave (nombre columna) – valor (función de agregación) o una lista de expresiones de agregación de Columna
*groupBy(*\**cols)* | Agrupa el DataFrame usando las columnas especificadas para poder aplicar agregaciones sobre los grupos – GroupedData




### `agg(*exprs)`
Agrega columnas numéricas en todo el DataFrame, sin grupos.

Agregaciones disponibles: avg, max, min, sum, count

\**exprs* puede ser

* Un diccionario clave (nombre columna) – valor (función de agregación)
+ Una lista de expresiones de agregación de Columna


In [36]:
print("Usando diccionario clave-valor: ")
df.agg({"age": "max"}).show()

from pyspark.sql import functions as F
print("Usando expresión de agregación en la columna:")
df.agg(F.min(df.age)).show()

Usando diccionario clave-valor: 
+--------+
|max(age)|
+--------+
|      25|
+--------+

Usando expresión de agregación en la columna:
+--------+
|min(age)|
+--------+
|       1|
+--------+



### `groupBy(*cols)`
Agrupa filas del DataFrame en base a las columnas especificadas

Se crea un DataFrame GroupedData, posteriormente se trata de aplicar agregaciones por grupos (ver siguiente sección)

In [37]:
print("Media usando groupBy().avg() - Se agrupa todo el DataFrame = usar agg() directamente")
print(df.groupBy().avg().collect())

print("\nAgrupamos por nombre y para cada nombre calculamos la edad media:")
print(sorted(df.groupBy('name').agg({'age': 'mean'}).collect()))
print(sorted(df.groupBy(df.name).avg().collect()))

print("\nAgrupamos por nombre y edad y contamos cuántos filas hay en cada grupo:")
sorted(df.groupBy(['name', df.age]).count().collect())

Media usando groupBy().avg() - Se agrupa todo el DataFrame = usar agg() directamente
[Row(avg(age)=11.0)]

Agrupamos por nombre y para cada nombre calculamos la edad media:
[Row(name='Alice', avg(age)=1.0), Row(name='Bob', avg(age)=4.0), Row(name='Juan', avg(age)=10.0), Row(name='Panchito', avg(age)=15.0), Row(name='Pepe', avg(age)=25.0)]
[Row(name='Alice', avg(age)=1.0), Row(name='Bob', avg(age)=4.0), Row(name='Juan', avg(age)=10.0), Row(name='Panchito', avg(age)=15.0), Row(name='Pepe', avg(age)=25.0)]

Agrupamos por nombre y edad y contamos cuántos filas hay en cada grupo:


[Row(name='Alice', age=1, count=1),
 Row(name='Bob', age=4, count=1),
 Row(name='Juan', age=10, count=1),
 Row(name='Panchito', age=15, count=1),
 Row(name='Pepe', age=25, count=1)]

### Transformaciones sobre GroupedData (groupBy)


Transformación | Descripción
------------- | -------------
*avg(*\**cols) / mean(*\**cols)* | Calcula la media de los valores para cada grupo en cada columna numérica
*count()* | Cuenta el número de registros (filas) en cada grupo
*max(**\**cols)* | Calcula el máximo valor para cada grupo en cada columna numérica
*min(*\**cols)* | Calcula el mínimo valor para cada grupo en cada columna numérica
*sum(*\**cols)* | Calcula la suma de los valores para cada grupo en cada columna numérica
*pivot(pivot_col, values)* | Pivota sobre una columna del DataFrame para realizar después la agregación especificada. Values especifica los valores que aparecerán en las columnas. Si no se especifica lo calcula Spark (menos eficiente)
*agg(*\**exprs)* | Realiza agregaciones sobre cada grupo del DataFrame. Agregaciones disponibles: avg, max, min, sum, count. exprs puede ser un diccionario clave (nombre columna) – valor (función de agregación) o una lista de expresiones de agregación de Columna

### `avg(*cols) / mean(*cols), count(), max(*cols), min(*cols), sum(*cols)`
Podemos aplicar directamente cualquiera de estos métodos sobre GroupedData para obtener el resultado de la función correspondiente por grupos

In [38]:
df2 = sc.parallelize([ \
    Row(name='Alice', age=5, height=80), \
    Row(name='Alice', age=5, height=80), \
    Row(name='Alice', age=10, height=80), \
    Row(name='Bob', age=5, height=90), \
    Row(name='Bob', age=10, height=100)]).toDF()

print("DataFrame ejemplo")
df2.show()

print("Edad media global con avg: " + str( df2.groupBy().avg('age').collect() ))
print("Edad y altura medias globales con avg: " + str( df2.groupBy().avg('age', 'height').collect() ))

print("Edad media global con mean" + str(df2.groupBy().mean('age').collect() ))
print("Edad y altura medias globales con mean" + str( df2.groupBy().mean('age', 'height').collect() ))


print("Media de edad y altura agrupoda por nombre: ")
df2.groupBy('name').avg('age', 'height').show()

print("Conteo de personas por edad:")
df2.groupBy(df2.age).count().show()

print("Edad máxima global" + str( df2.groupBy().max('age').collect() ) )
print("Edad y altura máximas globales" + str( df2.groupBy().max('age', 'height').collect() ))


print("Edad mínima global" + str( df2.groupBy().min('age').collect() ))
print("Edad y altura minima globales" + str( df2.groupBy().min('age', 'height').collect() ))

print("Suma global de edades" + str( df2.groupBy().sum('age').collect() ))
print("Suma global de edades y altura" + str( df2.groupBy().sum('age', 'height').collect() ))

DataFrame ejemplo
+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
|Alice|  5|    80|
|Alice| 10|    80|
|  Bob|  5|    90|
|  Bob| 10|   100|
+-----+---+------+

Edad media global con avg: [Row(avg(age)=7.0)]
Edad y altura medias globales con avg: [Row(avg(age)=7.0, avg(height)=86.0)]
Edad media global con mean[Row(avg(age)=7.0)]
Edad y altura medias globales con mean[Row(avg(age)=7.0, avg(height)=86.0)]
Media de edad y altura agrupoda por nombre: 
+-----+-----------------+-----------+
| name|         avg(age)|avg(height)|
+-----+-----------------+-----------+
|Alice|6.666666666666667|       80.0|
|  Bob|              7.5|       95.0|
+-----+-----------------+-----------+

Conteo de personas por edad:
+---+-----+
|age|count|
+---+-----+
|  5|    3|
| 10|    2|
+---+-----+

Edad máxima global[Row(max(age)=10)]
Edad y altura máximas globales[Row(max(age)=10, max(height)=100)]
Edad mínima global[Row(min(age)=5)]
Edad y altura minima globales[Row(min(age)=5, min

In [39]:
print("Edad media agrupado por nombre con avg: " )
df2.groupBy('name').avg('age').show()

print("Altura media agrupado por edad con avg: " )
df2.groupBy('age').avg('height').show()

print("Conteo de personas por altura:")
df2.groupBy(df2.height).count().show()

print("Edad máxima por nombre" )
df2.groupBy('name').max('age').show()

print("Edad media por nombre y altura" )
df2.groupBy('name', 'height').avg('age').show()

Edad media agrupado por nombre con avg: 
+-----+-----------------+
| name|         avg(age)|
+-----+-----------------+
|Alice|6.666666666666667|
|  Bob|              7.5|
+-----+-----------------+

Altura media agrupado por edad con avg: 
+---+-----------------+
|age|      avg(height)|
+---+-----------------+
|  5|83.33333333333333|
| 10|             90.0|
+---+-----------------+

Conteo de personas por altura:
+------+-----+
|height|count|
+------+-----+
|    80|    3|
|   100|    1|
|    90|    1|
+------+-----+

Edad máxima por nombre
+-----+--------+
| name|max(age)|
+-----+--------+
|Alice|      10|
|  Bob|      10|
+-----+--------+

Edad media por nombre y altura
+-----+------+-----------------+
| name|height|         avg(age)|
+-----+------+-----------------+
|Alice|    80|6.666666666666667|
|  Bob|   100|             10.0|
|  Bob|    90|              5.0|
+-----+------+-----------------+



### `agg(*exprs)`
Agrega columnas numéricas por grupos

Agregaciones disponibles: avg, max, min, sum, count

\**exprs* puede ser

* Un diccionario clave (nombre columna) – valor (función de agregación)
+ Una lista de expresiones de agregación de Columna


In [40]:
print("Conteo de personas con el mismo nombre:")
df2.agg({"*": "count"}).show()

Conteo de personas con el mismo nombre:
+--------+
|count(1)|
+--------+
|       5|
+--------+



In [41]:
from pyspark.sql import functions as F
print("Edad mínima para cada nombre:")
df2.groupBy(df2.name).agg(F.min(df2.age)).show()

Edad mínima para cada nombre:
+-----+--------+
| name|min(age)|
+-----+--------+
|Alice|       5|
|  Bob|       5|
+-----+--------+



In [42]:
data = [('Alice',1,6, 'Mate'), ('Bob',2,8, 'Mate'), ('Alice',3,9, 'Lengua'), ('Bob',4,7, 'Lengua')]
df = spark.createDataFrame(data, ['name', 'age', 'grade', 'subject'])

print("Media de edad y notas de todos los alumnos")
df.groupBy().avg().show()

print("Media de edad y notas de los alumnos con el mismo nombre")
df.groupBy('name').avg('age', 'grade').show()

print("Número de alumnos con el mismo nombre")
df1 = df.groupBy(df.name)
df1.agg({"*": "count"}).show() 
df.groupBy(df.name).count().show()

Media de edad y notas de todos los alumnos
+--------+----------+
|avg(age)|avg(grade)|
+--------+----------+
|     2.5|       7.5|
+--------+----------+

Media de edad y notas de los alumnos con el mismo nombre
+-----+--------+----------+
| name|avg(age)|avg(grade)|
+-----+--------+----------+
|Alice|     2.0|       7.5|
|  Bob|     3.0|       7.5|
+-----+--------+----------+

Número de alumnos con el mismo nombre
+-----+--------+
| name|count(1)|
+-----+--------+
|Alice|       2|
|  Bob|       2|
+-----+--------+

+-----+-----+
| name|count|
+-----+-----+
|Alice|    2|
|  Bob|    2|
+-----+-----+



### `pivot(pivot_col, values)`

Pivota sobre una columna del DataFrame para realizar después la agregación especificada

*values* especifica los valores que aparecerán en las columnas
* Si no se especifica lo calcula Spark (menos eficiente)
* Similar a pivot table de pandas


In [43]:
print("Notas medias por alumno y asignatura (modo eficiente especificando asignaturas): ")
df.groupBy("name").pivot("subject", ["Mate", "Lengua"]).avg("grade").show()

print("Notas medias por alumno y asignatura (modo no eficiente, dejando a Spark que busque los valores sobre los que pivotar): ")
df.groupBy("name").pivot("subject").avg("grade").show()

Notas medias por alumno y asignatura (modo eficiente especificando asignaturas): 
+-----+----+------+
| name|Mate|Lengua|
+-----+----+------+
|  Bob| 8.0|   7.0|
|Alice| 6.0|   9.0|
+-----+----+------+

Notas medias por alumno y asignatura (modo no eficiente, dejando a Spark que busque los valores sobre los que pivotar): 
+-----+------+----+
| name|Lengua|Mate|
+-----+------+----+
|  Bob|   7.0| 8.0|
|Alice|   9.0| 6.0|
+-----+------+----+



# Aspectos avanzados

## Funciones definidas por el usuario (User Defined Functions, UDF)

Podemos crear una función para trabajar sobre las columnas de los DataFrames mediante funciones definidas por el usuario. Sin embargo, debemos limitarnos a lo estríctamente necesario al ser mucho menos eficientes que las nativas de Spark.

In [44]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

print("Longitud con función definida por el usuario (UDF):")
slen = udf(lambda s: len(s), IntegerType())
df.select(slen(df.name).alias('slen')).show()

print("Longitud con versión de SparkSQL, mucho más eficiente:")
from pyspark.sql.functions import length
df.select(length(df.name).alias('length')).show()

Longitud con función definida por el usuario (UDF):


+----+
|slen|
+----+
|   5|
|   3|
|   5|
|   3|
+----+

Longitud con versión de SparkSQL, mucho más eficiente:
+------+
|length|
+------+
|     5|
|     3|
|     5|
|     3|
+------+



## SQL en Spark
Podemos trabajar directamente con SQL en Spark si registramos el DataFrame como tabla.

In [45]:
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+-----+---+-----+-------+
| name|age|grade|subject|
+-----+---+-----+-------+
|Alice|  1|    6|   Mate|
|  Bob|  2|    8|   Mate|
|Alice|  3|    9| Lengua|
|  Bob|  4|    7| Lengua|
+-----+---+-----+-------+



In [46]:
spark.sql("SELECT * FROM people WHERE age > 2 AND subject != 'Mate'").show()

+-----+---+-----+-------+
| name|age|grade|subject|
+-----+---+-----+-------+
|Alice|  3|    9| Lengua|
|  Bob|  4|    7| Lengua|
+-----+---+-----+-------+



## JOINs con SparkSQL

### Transformaciones Join tipo SQL

Transformación | Descripción
------------- | -------------
*join(other, on, how)*  | Joins entre dos DataFrames

* `other`: segunda tabla para el join
* `on`: nombre de la columna por la que se realiza la unión. Puede ser una lista o una expresión join (Column)
* `how`: tipo de unión entre inner, `outer, left_outer, right_outer, left_semi`

In [47]:
data = [Row(name=u'Alice', age=1), Row(name=u'Bob', age=2)]
data2 = [Row(name=u'Chris', height=80), Row(name=u'Bob', height=85)]

df = spark.createDataFrame(data, ['age', 'name'])
df2 = spark.createDataFrame(data2, ['height', 'name'])

print("df:")
df.show()

print("df2:")
df2.show()

df:
+-----+----+
|  age|name|
+-----+----+
|Alice|   1|
|  Bob|   2|
+-----+----+

df2:
+------+----+
|height|name|
+------+----+
| Chris|  80|
|   Bob|  85|
+------+----+



### `inner join`

In [48]:
print("Join por name")
df.join(df2, 'name').show()

print("Join por name + select df.name y height")
df.join(df2, 'name').select(df.name, df2.height).show()

Join por name
+----+---+------+
|name|age|height|
+----+---+------+
+----+---+------+

Join por name + select df.name y height
+----+------+
|name|height|
+----+------+
+----+------+



### `fullOuterJoin`

In [49]:
print("Full outer join por name")
df.join(df2, 'name', 'outer').show()

print("Full outer join por name + select df.name y height")
df.join(df2, 'name', 'outer').select('name', 'height').show()

Full outer join por name
+----+-----+------+
|name|  age|height|
+----+-----+------+
|   1|Alice|  null|
|   2|  Bob|  null|
|  80| null| Chris|
|  85| null|   Bob|
+----+-----+------+

Full outer join por name + select df.name y height
+----+------+
|name|height|
+----+------+
|   1|  null|
|   2|  null|
|  80| Chris|
|  85|   Bob|
+----+------+



### `leftOuterJoin`

In [50]:
print("Left outer join por name")
df.join(df2, 'name', 'left_outer').show()

Left outer join por name
+----+-----+------+
|name|  age|height|
+----+-----+------+
|   1|Alice|  null|
|   2|  Bob|  null|
+----+-----+------+



### `rightOuterJoin`

In [51]:
print("Left outer join por name")
df.join(df2, 'name', 'right_outer').show()

Left outer join por name
+----+----+------+
|name| age|height|
+----+----+------+
|  80|null| Chris|
|  85|null|   Bob|
+----+----+------+



### Más ejemplos

In [52]:
data = [('Alice',1,6, 'Mate'), \
        ('Bob',2,8, 'Mate'), \
        ('Alice',3,9, 'Lengua'), \
        ('Bob',4,7, 'Lengua')]

df = spark.createDataFrame(data, ['name', 'course', 'grade', 'subject'])
df2 = sc.parallelize([ \
    Row(name='Alice', age=5, height=80), \
    Row(name='Alice', age=5, height=80), \
    Row(name='Alice', age=10, height=80), \
    Row(name='Bob', age=5, height=90), \
    Row(name='Bob', age=10, height=100), \
    Row(name='Juan', age=5, height=90), \
    Row(name='Jaimito', age=6, height=120)]).toDF()

print("df:")
df.show()

print("df2:")
df2.show()

df:
+-----+------+-----+-------+
| name|course|grade|subject|
+-----+------+-----+-------+
|Alice|     1|    6|   Mate|
|  Bob|     2|    8|   Mate|
|Alice|     3|    9| Lengua|
|  Bob|     4|    7| Lengua|
+-----+------+-----+-------+

df2:
+-------+---+------+
|   name|age|height|
+-------+---+------+
|  Alice|  5|    80|
|  Alice|  5|    80|
|  Alice| 10|    80|
|    Bob|  5|    90|
|    Bob| 10|   100|
|   Juan|  5|    90|
|Jaimito|  6|   120|
+-------+---+------+



In [53]:
print("Inner join usando name")
df.join(df2, 'name').show()

print("Outer join usando name + select name y height")
df.join(df2, 'name', 'outer').select('name', 'height').show()

print("Outer join usando df.name == df2.name + select df.name y height")
df.join(df2, df.name == df2.name, 'outer').select(df.name, df2.height).show()

print("Outer join usando condición con name y age = grade (a pesar de no tener sentido) + select name y age")
cond = [df.name == df2.name, df2.age == df.grade]
df.join(df2, cond, 'outer').select(df.name, df2.age).show()

Inner join usando name
+-----+------+-----+-------+---+------+
| name|course|grade|subject|age|height|
+-----+------+-----+-------+---+------+
|Alice|     1|    6|   Mate|  5|    80|
|Alice|     1|    6|   Mate|  5|    80|
|Alice|     1|    6|   Mate| 10|    80|
|Alice|     3|    9| Lengua|  5|    80|
|Alice|     3|    9| Lengua|  5|    80|
|Alice|     3|    9| Lengua| 10|    80|
|  Bob|     2|    8|   Mate|  5|    90|
|  Bob|     2|    8|   Mate| 10|   100|
|  Bob|     4|    7| Lengua|  5|    90|
|  Bob|     4|    7| Lengua| 10|   100|
+-----+------+-----+-------+---+------+

Outer join usando name + select name y height
+-------+------+
|   name|height|
+-------+------+
|  Alice|    80|
|  Alice|    80|
|  Alice|    80|
|  Alice|    80|
|  Alice|    80|
|  Alice|    80|
|    Bob|    90|
|    Bob|   100|
|    Bob|    90|
|    Bob|   100|
|Jaimito|   120|
|   Juan|    90|
+-------+------+

Outer join usando df.name == df2.name + select df.name y height
+-----+------+
| name|height|
+--

# Caché de DataFrames
Si se va a reusar un DataFrame es conveniente cachearlo, así no se recalcula

In [54]:
quijoteDF = spark.read.text("./datos/pg2000.txt")
palabrasQuijoteDF = quijoteDF.select(explode(split('value', ' ')).alias('word')).cache()
print("Cabeza aparece " + str( palabrasQuijoteDF.where(palabrasQuijoteDF.word.like("%cabeza%")).count() ) + " veces")
print("Lanza aparece " + str( palabrasQuijoteDF.where(palabrasQuijoteDF.word.like("%lanza%")).count() ) + " veces")

Cabeza aparece 230 veces
Lanza aparece 90 veces


# Lectura y escritura de ficheros


## Lectura
Leer ficheros de texto, JSON o CVS es muy sencillo.

Hay dos formas de hacerlo:
* Usando `spark.read.tipo_fichero(fichero)`
* Usando `spark.read.format(tipo_fichero).options(opciones).load(fichero)`

In [57]:
dfText = spark.read.text("datos/pg2000.txt")
dfJSON = spark.read.json("datos/json.json") # infiere el esquema
dfCSV = spark.read.csv("datos/personas.csv", inferSchema=True, header=True)

print("Elementos en DataFrame a partir de datos/pg2000.txt: " + str(dfText.count()) + "\nEsquema: ")
print (dfText.printSchema())

print("Elementos en DataFrame a partir de datos/json.json: " + str(dfJSON.count()) + "\nEsquema: ")
print (dfJSON.printSchema())

print("Elementos en DataFrame a partir de datos/personas.csv: " + str(dfCSV.count()) + "\nEsquema: ")
print (dfCSV.printSchema())

Elementos en DataFrame a partir de datos/pg2000.txt: 37861
Esquema: 
root
 |-- value: string (nullable = true)

None
Elementos en DataFrame a partir de datos/json.json: 5
Esquema: 
root
 |-- age: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

None
Elementos en DataFrame a partir de datos/personas.csv: 100
Esquema: 
root
 |-- Persona: string (nullable = true)
 |-- Tel: string (nullable = true)
 |-- email: string (nullable = true)

None


In [58]:
dfText = spark.read.format("text").load("datos/pg2000.txt")
dfJSON = spark.read.format("json").load("datos/json.json") # infiere el esquema
dfCSV = spark.read.format("csv").options(inferSchema=True, header=True).load("datos/personas.csv")

print ("Elementos en DataFrame a partir de datos/pg2000.txt: " + str(dfText.count()) + "\nEsquema: ")
print (dfText.printSchema())

print ("Elementos en DataFrame a partir de datos/json.json: " + str(dfJSON.count()) + "\nEsquema: ")
print (dfJSON.printSchema())

print ("Elementos en DataFrame a partir de datos/personas.csv: " + str(dfCSV.count()) + "\nEsquema: ")
print (dfCSV.printSchema())

Elementos en DataFrame a partir de datos/pg2000.txt: 37861
Esquema: 
root
 |-- value: string (nullable = true)

None
Elementos en DataFrame a partir de datos/json.json: 5
Esquema: 
root
 |-- age: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

None
Elementos en DataFrame a partir de datos/personas.csv: 100
Esquema: 
root
 |-- Persona: string (nullable = true)
 |-- Tel: string (nullable = true)
 |-- email: string (nullable = true)

None


## Escritura
Escribir ficheros de texto, JSON o CVS es igual de fácil.

**Nota: El fichero de salida se toma como directorio**

Hay dos formas de hacerlo:
* Usando `DataFrame.write.tipo_fichero(fichero)`
* Usando `DataFrame.write.format(tipo_fichero).options(opciones).load(fichero)`

In [59]:
dfText.write.text("datos/salidaTXT1.txt")
dfJSON.write.json("datos/salidaJSON1.json")
dfCSV.write.csv("datos/salidaCSV1.csv", header=True)

print("Ver datos escritos en datos/salidaTXT1.txt")
print("Ver datos escritos en datos/salidaJSON1.json")
print("Ver datos escritos en datos/salidaCSV1.csv")

Ver datos escritos en datos/salidaTXT1.txt
Ver datos escritos en datos/salidaJSON1.json
Ver datos escritos en datos/salidaCSV1.csv


In [60]:
## ¡¡CUIDADO FALLA SI EXISTE!!
dfText.write.format('text').save("datos/salidaTXT2.txt")
dfJSON.write.format('json').save("datos/salidaJSON2.json")
dfCSV.write.format('csv').save("datos/salidaCSV2.csv", header=True)

print("Ver datos escritos en datos/salidaTXT2.txt")
print("Ver datos escritos en datos/salidaJSON2.json")
print("Ver datos escritos en datos/salidaCSV2.csv")

AnalysisException: path file:/home/alumno/Descargas/P2/datos/salidaTXT2.txt already exists.

## Más opciones en: <http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter>

# Un ejemplo completo: WordCount simple con DataFrames

*value* es el nombre de la columna por defecto al leer texto

1. Aplicamos un select con explode que equivale a un flatMap
1. Dentro del select dividimos la línea por espacios con split
 * A la columna resultante la denominamos Word con alias
 * Filtramos palabras vacías
2. Agrupamos por palabra y contamos cuántas veces se repite cada una
3. Ordenamos de manera descendente
4. Mostramos el DataFrame obtenido


In [61]:
from pyspark.sql.functions import *
dfText = spark.read.format("text").load("datos/pg2000.txt")
dfText.select(explode(split(dfText["value"], " ")).alias("word")) \
    .filter("word != ''") \
    .groupBy("word").count() \
    .sort(desc("count")) \
    .show()

+----+-----+
|word|count|
+----+-----+
| que|19429|
|  de|17988|
|   y|15894|
|  la|10200|
|   a| 9575|
|  el| 7957|
|  en| 7898|
|  no| 5611|
|  se| 4690|
| los| 4680|
| con| 4047|
| por| 3758|
| las| 3423|
|  lo| 3387|
|  le| 3382|
|  su| 3319|
| don| 2533|
| del| 2464|
|  me| 2344|
|como| 2226|
+----+-----+
only showing top 20 rows



## Por pasos

In [62]:
dfText.cache()

DataFrame[value: string]

#### Vemos lo que hay en el DataFrame

In [63]:
dfText.select("*").show(10, False)

+---------------------------------------------------------------------------+
|value                                                                      |
+---------------------------------------------------------------------------+
|The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra|
|                                                                           |
|This eBook is for the use of anyone anywhere at no cost and with           |
|almost no restrictions whatsoever.  You may copy it, give it away or       |
|re-use it under the terms of the Project Gutenberg License included        |
|with this eBook or online at www.gutenberg.net                             |
|                                                                           |
|                                                                           |
|Title: Don Quijote                                                         |
|                                                               

#### Dividimos las líneas en palabras

In [64]:
dfText.select(split("value", " ")) \
      .show(10, False)

+----------------------------------------------------------------------------------------+
|split(value,  , -1)                                                                     |
+----------------------------------------------------------------------------------------+
|[The, Project, Gutenberg, EBook, of, Don, Quijote,, by, Miguel, de, Cervantes, Saavedra]|
|[]                                                                                      |
|[This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]         |
|[almost, no, restrictions, whatsoever., , You, may, copy, it,, give, it, away, or]      |
|[re-use, it, under, the, terms, of, the, Project, Gutenberg, License, included]         |
|[with, this, eBook, or, online, at, www.gutenberg.net]                                  |
|[]                                                                                      |
|[]                                                                                      |

#### Usamos explode para crear una Row con cada elemento del array en la columna actual

In [65]:
dfText.select( \
       explode(split("value", " "))) \
      .show(10)

+---------+
|      col|
+---------+
|      The|
|  Project|
|Gutenberg|
|    EBook|
|       of|
|      Don|
| Quijote,|
|       by|
|   Miguel|
|       de|
+---------+
only showing top 10 rows



#### Renombramos la nueva columna


In [66]:
dfText.select( \
       explode(split("value", " ")) \
      .alias("word")) \
      .show(10)

+---------+
|     word|
+---------+
|      The|
|  Project|
|Gutenberg|
|    EBook|
|       of|
|      Don|
| Quijote,|
|       by|
|   Miguel|
|       de|
+---------+
only showing top 10 rows



#### Filtramos palabras vacías


In [67]:
dfText.select( \
       explode(split("value", " ")) \
      .alias("word")) \
      .filter("word != ''") \
      .show(10)

+---------+
|     word|
+---------+
|      The|
|  Project|
|Gutenberg|
|    EBook|
|       of|
|      Don|
| Quijote,|
|       by|
|   Miguel|
|       de|
+---------+
only showing top 10 rows



#### Agrupamos por palabra y contamos el número de apariciones


In [68]:
dfText.select( \
       explode(split("value", " ")) \
      .alias("word")) \
      .filter("word != ''") \
      .groupBy("word").count() \
      .show(10)

+------------+-----+
|        word|count|
+------------+-----+
|      online|    4|
|      saben,|    3|
|      Platón|    1|
|     tempora|    1|
|Comentarios,|    1|
|       fuere|   64|
|       ense-|    1|
|          tú|  185|
|      gloria|   35|
|      formó,|    1|
+------------+-----+
only showing top 10 rows



#### Ordenamos de manera descendente por la cuenta


In [69]:
dfText.select( \
       explode(split("value", " ")) \
      .alias("word")) \
      .filter("word != ''") \
      .groupBy("word").count() \
      .sort(desc("count")) \
      .show(10)

+----+-----+
|word|count|
+----+-----+
| que|19429|
|  de|17988|
|   y|15894|
|  la|10200|
|   a| 9575|
|  el| 7957|
|  en| 7898|
|  no| 5611|
|  se| 4690|
| los| 4680|
+----+-----+
only showing top 10 rows

